In [5]:
import math

from langchain_core.tools import tool
from langchain_community.utilities import SQLDatabase
from app.config import database
from core.chains.sql.utils import get_db
import urllib
from langchain_openai import ChatOpenAI
import json
import os
from dotenv import load_dotenv
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

include_tables = {"rpt_customer_profile", "rpt_order_details_report"}

mysql_user = os.getenv("DB_USER")
mysql_password = urllib.parse.quote_plus(os.getenv("DB_PASSWORD"))
mysql_host = os.getenv("DB_HOST")
encoded_password = (mysql_password)
mysql_uri = f"mysql+pymysql://{mysql_user}:{encoded_password}@{mysql_host}:3306/vnv"
db = SQLDatabase.from_uri(mysql_uri, {}, include_tables=include_tables)
# 
# llm = ChatOpenAI(model="gpt-4o", temperature=0)
# create_sql_agent(llm, db=db, verbose=True)
toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(model="gpt-4o"))
tools = toolkit.get_tools()

@tool
def format_in_indian_currency(amount):
    """Converts a numeric amount into Indian rupee format with Lakhs/Crores."""
    amount = math.ceil(amount)
    if amount >= 10000000:  # 1e7 = 1,00,00,000
        return f"{amount / 10000000:.2f} Crores"
    elif amount >= 100000:  # 1e5 = 1,00,000
        return f"{amount / 100000:.2f} Lakhs"
    else:
        return f"₹{amount:,}"
    
tools.extend([format_in_indian_currency])
# print(tools)
# list_tables_tool = next(tool for tool in tools if tool.name == "sql_db_list_tables")
# print(list_tables_tool.invoke(""))
# get_schema_tool = next(tool for tool in tools if tool.name == "sql_db_schema")
# print(get_schema_tool.invoke("rpt_customer_profile"))

In [8]:
from langgraph.prebuilt import create_react_agent
from langgraph.graph import add_messages
from langchain_core.messages import AnyMessage
from typing import TypedDict, Annotated

prompt = """You are a SQL expert with a strong attention to detail.
Given an input question, output a syntactically correct MySql query to run, then look at the results of the query and return the answer using the guidelines mentioned below.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
If you get an error while executing a query, rewrite the query and try again.
If you get an empty result set, you should try to rewrite the query to get a non-empty result set.
NEVER make stuff up if you don't have enough information to answer the query... just say you don't have enough information.
If you have enough information to answer the input question, provide a concise final answer to the user.
DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

Guidelines for the final answer:
1. Keep the tone conversational and friendly.
2. Analyse the result data and try to provide a trend or analysis if possible.
3. Provide suggestions on similar questions related to the question asked.
"""

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    
llm = ChatOpenAI(model="gpt-4o", temperature=0)
# model = llm.bind_tools(tools)

graph = create_react_agent(llm, tools, state_modifier=prompt)
inputs = {"messages": [("user", "most selling products in  2023")]}
for s in graph.stream(inputs, stream_mode="values"):
     message = s["messages"][-1]
     if isinstance(message, tuple):
         print(message)
     else:
         message.pretty_print()



================================ Human Message =================================

most selling products in  2023
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_RR4QJa2hVduH2GR8WM4ceY7d)
 Call ID: call_RR4QJa2hVduH2GR8WM4ceY7d
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

rpt_customer_profile, rpt_order_details_report
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_eMo1oK0UPEXskgkBwRrQFtpY)
 Call ID: call_eMo1oK0UPEXskgkBwRrQFtpY
  Args:
    table_names: rpt_order_details_report
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE rpt_order_details_report (
	order_id INTEGER UNSIGNED NOT NULL DEFAULT '0', 
	customer_id INTEGER UNSIGNED, 
	first_name VARCHAR(191) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci, 
	last_n